<a href="https://colab.research.google.com/github/azhary86/bigdata2023/blob/main/MidTerm_BigData2023_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')
# files.upload()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
import pandas as pd
# data = pd.read_csv('/content/gdrive/My Drive/bigdata2023/market-basket.csv',on_bad_lines='skip')
# data_e = data.to_excel('/content/gdrive/My Drive/bigdata2023/market-basket.xlsx', index=None, header=True)
data = pd.read_excel('/content/gdrive/My Drive/bigdata2023/market-basket.xslx')

In [30]:
data.head()

AttributeError: ignored

In [26]:
data1 = data[['BIllNo','Itemname','CustomerID']]
data1.head()

KeyError: ignored

In [ ]:
df= data1.groupBy('CustomerID') ['Itemname  '].apply(lambda x: x.reset_index (drop = True)). unstack (). reset_index ())
df.drop('CustomerID',axis=1, inplace=True)
df.head()

In [ ]:
items = (df[0].unique())
items

In [ ]:
encoded_vals = []
def custom():
    for index, row in df.iterrows():
        labels = {}
        uncommons = list(set(items) - set(row))
        commons = list(set(items).intersection(row))
        for uc in uncommons:
            labels[uc] = 0
        for com in commons:
            labels[com] = 1
        encoded_vals.append(labels)
custom()
ohe_df = pd.DataFrame(encoded_vals)
print(ohe_df)

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
freq_items = apriori(ohe_df, min_support=0.006, use_colnames=True) #, verbose=1 #min_support=0.006, 
freq_items

In [ ]:
freq_items.to_excel('Export&Supportvalues.xlsx')
rules = association_rules(freq_items, metric="confidence", min_threshold=0.006)
rules

In [7]:
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# !mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark

In [ ]:
from google.colab import files
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth
import pandas

sparkdata = spark.createDataFrame(data)
basketdata = sparkdata.dropDuplicates(['CustomerID', 'Itemname']).sort('CustomerID')
basketdata = basketdata.groupBy("CustomerID").agg(F.collect_list("Itemname")).sort('CustomerID')

In [ ]:
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.006, minConfidence=0.006) 
model = fpGrowth.fit(basketdata)

model.freqItemsets.show()
items = model.freqItemsets
model.associationRules.show()
rules = model.associationRules
model.transform(basketdata).show()
transformed = model.transform(basketdata)

In [ ]:
result_pdf = items.select("*").toPandas()
result_pdf.head()

In [ ]:
result_pdf.to_excel('resultItemsFreq.xlsx')
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

# **Conclusion:**


*    
*    

